In [1]:
import re
from underthesea import word_tokenize
from rank_bm25 import BM25Okapi, BM25Plus
import pyvi
from pyvi import ViTokenizer, ViPosTagger
import pickle

In [3]:
file = open('../data.txt',encoding='utf-8').read().split("\n")
data = []
for line in file:
    if re.search("^.*\.", line) or re.search("^.*\)", line):
        data.append(line.lower())

stopwords = open('../punctual.txt', encoding='utf-8').read().split(" ")

In [4]:
def removeStopwords(sentence):
  words = word_tokenize(sentence)
  word = [w.lower() for w in words if w not in (stopwords)]
  sentence_clean = " ".join(word)
  
  return sentence_clean

In [5]:
def removeDash(sentence):
    words = sentence.split('_')
    return " ".join(words)

In [6]:
def cleanQuestion(qs):
    getLabels = ['N','M','V','Np','A']
    posTagedQS =  ViPosTagger.postagging(ViTokenizer.tokenize(qs))
    cleanedQS = []
    for i in range(len(posTagedQS[1])):
        if posTagedQS[1][i] in getLabels:
            cleanedQS.append(posTagedQS[0][i])
    
    result = " ".join(cleanedQS)
    return removeStopwords(removeDash(result))
    

In [7]:
def bm25okapi_search(tokenized_query, bm25, corpus, n_results = 1):
    """
    Function that takes a tokenized query and prints the first 100 words of the 
    n_results most relevant results found in the corpus, based on the BM25
    method.
    
    Parameters
    ----------
    @param tokenized_query: list, array-like
        A valid list containing the tokenized query.
    @param bm25: BM25 object,
        A valid object of type BM25 (BM25Okapi or BM25Plus) from the library
        `rank-bm25`, initialized with a valid corpus.
    @param corpus: list, array-like
        A valid list containing the corpus from which the BM25 object has been 
        initialized. As returned from function read_corpus().
    @param n_results: int, default = 1
        The number of top results to print.
    """
    
    # We skip checking validity of arguments for now... We assume the user 
    # knows what they're doing.
    
    # Get top results for the query
    top_results = bm25.get_top_n(tokenized_query, corpus, n = n_results)
    top_results_100words = [' '.join(top_result.split(' ')) 
                             for top_result in top_results]
    
    return top_results_100words

In [8]:
tokenized_data = [ViTokenizer.tokenize(doc) for doc in data]


In [9]:
bm25 = BM25Okapi(tokenized_data)

In [51]:
# Save model to load
with open('bm25result', 'wb') as bm25result_file:
    pickle.dump(bm25, bm25result_file)

In [52]:
#to read bm25 object
with open('bm25result', 'rb') as bm25result_file:
    bm25result = pickle.load(bm25result_file)

In [16]:
query = "khi nào tòa án có thể ra quyết định tuyên bố một người bị mất năng lực hành vi dân sự ?"
tokenized_query = ViTokenizer.tokenize(cleanQuestion(query))
print(tokenized_query)
bm25okapi_search(tokenized_query = tokenized_query,
                 bm25 = bm25, 
                 corpus = data,
                 n_results = 5)

khi tòa_án ra quyết_định tuyên_bố một người bị mất năng_lực hành_vi dân_sự


['khi không còn căn cứ tuyên bố một người mất năng lực hành vi dân sự thì theo yêu cầu của chính người đó hoặc của người có quyền, lợi ích liên quan hoặc của cơ quan, tổ chức hữu quan, tòa án ra quyết định huỷ bỏ quyết định tuyên bố mất năng lực hành vi dân sự.',
 '3. khi không còn căn cứ tuyên bố một người bị hạn chế năng lực hành vi dân sự thì theo yêu cầu của chính người đó hoặc của người có quyền, lợi ích liên quan hoặc của cơ quan, tổ chức hữu quan, tòa án ra quyết định huỷ bỏ quyết định tuyên bố hạn chế năng lực hành vi dân sự.',
 'b) người giám hộ là cá nhân chết hoặc bị tòa án tuyên bố hạn chế năng lực hành vi dân sự, có khó khăn trong nhận thức, làm chủ hành vi, mất năng lực hành vi dân sự, mất tích; pháp nhân làm giám hộ chấm dứt tồn tại;',
 'b) người chưa thành niên có cha, mẹ nhưng cha, mẹ đều mất năng lực hành vi dân sự; cha, mẹ đều có khó khăn trong nhận thức, làm chủ hành vi; cha, mẹ đều bị hạn chế năng lực hành vi dân sự; cha, mẹ đều bị tòa án tuyên bố hạn chế quyền đối